<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Meta_Llama_3_8B_for_MEDAL_EVALUATOR-cosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip show huggingface_hub

In [2]:
#!pip install --upgrade huggingface_hub -q

In [3]:
#!pip install huggingface_hub -q

https://medium.com/@frankmorales_91352/fine-tuning-meta-llama-3-8b-with-medal-for-enhanced-medical-language-understanding-3de07a54fab9

In [ ]:
! pip install peft --quiet

!pip install -U flash-attn --no-build-isolation --quiet

#Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

In [5]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

In [6]:
!pip install colab-env --quiet

  Preparing metadata (setup.py) ... done


In [7]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Mounted at /content/gdrive


In [8]:
from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
#!pip install -U tensorboard-plugin-profile -q

In [10]:
%rm -rf /content/tensorbard
%mkdir -p /content/tensorbard
%cd /content/tensorbard
!git lfs install
#!git clone https://huggingface.co/frankmorales2020/Meta-Llama-3-8B-MEDAL-flash-attention-2
!git clone https://huggingface.co/frankmorales2020/Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine

/content/tensorbard
Git LFS initialized.
Cloning into 'Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 71 (delta 10), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (71/71), 2.24 MiB | 3.26 MiB/s, done.
Filtering content: 100% (3/3), 1.25 GiB | 20.82 MiB/s, done.


In [ ]:
%load_ext tensorboard
#%tensorboard --logdir /content/tensorbard/Meta-Llama-3-8B-MEDAL-flash-attention-2/runs
#%tensorboard --logdir /content/gdrive/MyDrive/model/Meta-Llama-3-8B-MEDAL-flash-attention-2/runs/

%tensorboard --logdir /content/tensorbard/Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine/runs
#%tensorboard --logdir /content/gdrive/MyDrive/model/Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine/runs/


In [12]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

#peft_model_id = "/content/gdrive/MyDrive/model/Meta-Llama-3-8B-MEDAL-flash-attention-2"
#peft_model_id = 'frankmorales2020/Meta-Llama-3-8B-MEDAL-flash-attention-2'

#peft_model_id = "/content/gdrive/MyDrive/model/Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine"
peft_model_id = 'frankmorales2020/Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine'


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
  #load_in_4bit=True
  quantization_config=bnb_config
)

#model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf',  load_in_4bit=True, ...)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# load into pipeline
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

adapter_config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
from datasets import load_dataset
from random import randint


# Load our test dataset
#/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
#nrec= randint(0, len(eval_dataset))

nrec=6

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['text']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [15]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
ganswer

'for the management of this debilitating condition [/INST] antral follicle count </s><s>[INST] the purpose of this study was to investigate the effect of the novel antifolate aminopterin on the survival of mu and human glioma cells in vitro and in vivo aminopterin was more effective than methotrexate mtx in inhibiting the growth of a murine glioma cell L1 in vitro the ic for aminopterin was much lower than that of mtx nm vs nm aminopterin also inhibited the growth of a human glioma cell L1 in vitro the ic was nm the survival'

In [16]:
def similarity(ganswer,oanswer):

  qc0=str(ganswer).find('[INST]')

  ganswer=str(ganswer)[0:qc0-8]
  qc=str(ganswer).find('[/INST]')
  if qc>0:
    ganswer=ganswer[qc+8:len(ganswer)]

  print(f"Generated Answer:\n{ganswer}")

  if ganswer == oanswer:
    print("Match")
    match=1
  else:
    print("NO Match")
    match=0
  return match

In [17]:
print(f"Query:\n{eval_dataset[nrec]['text']}")

oanswer=str(eval_dataset[nrec]['label'])
oanswer=oanswer[2:len(oanswer)-2]
print(f"Original Answer:\n{oanswer}")

ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()

match=similarity(ganswer,oanswer)

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [18]:
nrec=6
print(f"Query:\n{eval_dataset[nrec]['text']}")
print(f"Original Answer:\n{eval_dataset[nrec]['label']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [19]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc=str(ganswer).find('[/INST]')
#print(qc)
if int(qc)<0:
    #print(f"Generated Answer:\n{ganswer}")
    ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
else:
    ganswer=ganswer[qc+8:len(ganswer)]
ganswer

'antral follicle count </s><s>[INST] the purpose of this study was to investigate the effect of the novel antifolate aminopterin on the survival of mu and human glioma cells in vitro and in vivo aminopterin was more effective than methotrexate mtx in inhibiting the growth of a murine glioma cell L1 in vitro the ic for aminopterin was much lower than that of mtx nm vs nm aminopterin also inhibited the growth of a human glioma cell L1 in vitro the ic was nm the survival'

In [20]:
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [21]:
def exceptions(ganswer,cganswer,eganswer):
     if ganswer==eganswer:
        ganswer=cganswer
     return ganswer

In [27]:
from tqdm import tqdm

#prompt =  eval_dataset[nrec]['text']


#outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
#                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
#                                  pad_token_id=tokenizer.eos_token_id)


def evaluate(sample):
    prompt =  sample['text']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

    ganswer=outputs[0]['generated_text'][len(prompt):].strip()

    #print()
    #print(f"Question:\n{sample['text']}")
    #print()

    #print()
    #print(f"Generated Answer original:\n{ganswer}")
    #print()

    # molecule [/INST] phosphorylethanolamine </s><s>[INST]
    qc0=str(ganswer).find('[/INST]')
    qc1=str(ganswer).find('[INST]')

    #print(qc0)
    #print(qc1)

    ganswer=str(ganswer)[qc0+8:qc1-8]
    #print(f"Generated Answer corr:\n{ganswer}")

    oanswer=str(sample['label'])
    oanswer=oanswer[2:len(oanswer)-2]


    #### exceptions TODO

    #if ganswer == 'enteropathogenic':
    #   ganswer=exceptions(ganswer,oanswer,ganswer)


    #print(len(oanswer))
    #print(len(ganswer))

    #print(f"Original Answer:\n{oanswer}")
    if ganswer == oanswer:
        #print()
        #print("Match")
        #print(f"Query:\n{sample['text']}")
        #print(f"Original Answer:\n{oanswer}")
        #print(f"Generated Answer:\n{ganswer}")
        #print()
        return 1
    else:
        #print()
        #print("no Match")
        #print(f"Query:\n{sample['text']}")
        #print(f"Original Answer:\n{oanswer}")
        #print(f"Generated Answer:\n{ganswer}")
        #print()
        return 0


#### exceptions TODO

#Original Answer:
#enteropathogenic
#Generated Answer:
#enteropathogenic e coli

#Original Answer:
#severe mental retardation
#Generated Answer:
#mental retardation

#Original Answer:
#eye movement
#Generated Answer:
#eye movements


#Original Answer:
#severe mental retardation
#Generated Answer:
#mental retardation


success_rate = []
number_of_eval_samples = 10
# iterate over eval dataset and predict

#for n in tqdm(range(number_of_eval_samples)):
for n in tqdm(range(number_of_eval_samples)):
#    print(f'sample {n}')
    s=eval_dataset[n]
    success_rate.append(evaluate(s))

#for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
#    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

100%|██████████| 10/10 [03:06<00:00, 18.68s/it]


In [28]:
#print(f"Accuracy: {accuracy*100:.2f}%")

print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

Accuracy (Eval dataset and predict) for a sample of 10: 60.00%



learning_rate: 2e-05
train_batch_size: 3
eval_batch_size: 8
seed: 42
gradient_accumulation_steps: 6
total_train_batch_size: 18
optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
lr_scheduler_type: constant
lr_scheduler_warmup_ratio: 0.03
num_epochs: 2

When evaluated on 1000 samples from the evaluation dataset, our model achieved an accuracy of 26.30%. However, there's room for improvement.


learning_rate: 0.0002
train_batch_size: 3
eval_batch_size: 8
seed: 42
gradient_accumulation_steps: 6
total_train_batch_size: 18
optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
lr_scheduler_type: cosine
lr_scheduler_warmup_ratio: 0.03
num_epochs: 10

When evaluated on 10 samples from the evaluation dataset, our model achieved an accuracy of 60%. However, there's room for improvement.

